In [1]:
# Importing pacakges
import sys

sys.path.append('..')

import numpy as np

from src.pychange.online import OnlineCP, StudentTProb, ConstantHazard
from src.pychange.r import ROCP

Unable to determine R home: [WinError 2] The system cannot find the file specified
Unable to determine R library path: Command '('C:\\Program Files\\R\\R-4.1.1\\bin\\Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 1.


In [2]:
# Creating synthetic data
data = np.hstack([
    np.random.normal(0, 1, (1000,)),
    np.random.normal(10, 1, (1000,)),
    np.random.normal(0, 1, (1000,)),
    np.random.normal(10, 1, (1000,)),
    ])

In [ ]:
# Using R
class ROCP:

    def __init__(self, **kwargs):
        self.kwargs = kwargs
    
    def fit(self, signal):
        _x = robjects.FloatVector(signal)
        self.cp = rocp.onlineCPD(_x)
        return self
    
    def predict(self):
        return np.array(self.cp.rx2('changepoint_lists').rx2('colmaxes'))[0].astype(np.int64)[1:]


In [3]:
ROCP().fit(data).predict()

array([1001, 2001, 2733, 3001, 4001, 5001, 6001, 7001, 8001, 9001],
      dtype=int64)

In [3]:
OnlineCP(ConstantHazard(), StudentTProb()).update(data).get_cps()

[1000, 2000, 3000]

In [3]:
%timeit ROCP().fit(data).predict()

Exception ignored from cffi callback <function _callback at 0x000001A4724E7670>:
Traceback (most recent call last):
  File "C:\Users\15072\anaconda3\envs\pychange\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 329, in _callback
    try:
KeyboardInterrupt: 


In [ ]:
%timeit OnlineCP(ConstantHazard(), StudentTProb()).update(data).get_cps()

354 ms ± 702 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
from time import time

def make_data(n):
    return np.hstack([
    np.random.normal(0, 1, (n,)),
    np.random.normal(10, 1, (n,)),
    np.random.normal(0, 1, (n,)),
    np.random.normal(10, 1, (n,)),
    np.random.normal(0, 1, (n,)),
    np.random.normal(10, 1, (n,)),
    np.random.normal(0, 1, (n,)),
    np.random.normal(10, 1, (n,)),
    np.random.normal(0, 1, (n,)),
    np.random.normal(10, 1, (n,)),
    ])
for n in [30, 50, 100, 150, 250, 500, 750]:
    data = make_data(n)
    py_times = []
    r_times = []
    # Python
    for _ in range(5):
        start_time = time()
        _ = OnlineCP(ConstantHazard(), StudentTProb()).update(data).get_cps()
        py_times.append(time() - start_time)
    # R
    for _ in range(5):
        start_time = time()
        _ = ROCP().fit(data).predict()
        r_times.append(time() - start_time)
    
    print(n, np.mean(py_times).round(5), np.mean(r_times).round(5), round(np.mean(r_times) / np.mean(py_times), 3))

30 0.0132 0.14 10.607
50 0.03 0.26145 8.715
100 0.09947 0.67179 6.754
150 0.20815 1.24109 5.962
250 0.54325 2.8278 5.205
500 2.37708 9.75038 4.102
750 6.88845 21.16839 3.073
